# Video Games JP_Sales: исправленная версия (Name + franchise features)


In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

from scipy import sparse
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge

from catboost import CatBoostRegressor

RANDOM_STATE = 42
TRAIN_PATH = "external.csv"
TEST_PATH  = "Video_Games_Test.csv"
TARGET = "JP_Sales"

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

# на случай "Id " / " id" / пробелов
train_df.columns = train_df.columns.str.strip()
test_df.columns  = test_df.columns.str.strip()

assert TARGET in train_df.columns, f"'{TARGET}' not found in train"

y = train_df[TARGET].astype(float)
X_train_raw = train_df.drop(columns=[TARGET]).copy()
X_test_raw  = test_df.copy()

def find_id_col(df):
    for c in df.columns:
        if str(c).strip().lower() == "id":
            return c
    return None

id_col_train = find_id_col(X_train_raw)
id_col_test  = find_id_col(X_test_raw)

if id_col_test is not None:
    test_ids = X_test_raw[id_col_test].values
else:
    test_ids = np.arange(1, len(X_test_raw) + 1)

if id_col_train is not None:
    X_train_raw.drop(columns=[id_col_train], inplace=True)
if id_col_test is not None:
    X_test_raw.drop(columns=[id_col_test], inplace=True)

X_all = pd.concat([X_train_raw, X_test_raw], axis=0, ignore_index=True)

print("Train rows:", len(X_train_raw), "Test rows:", len(X_test_raw), "All rows:", len(X_all))
print("Columns:", list(X_all.columns))

Train rows: 16719 Test rows: 5016 All rows: 21735
Columns: ['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating']


In [2]:
print(X_train_raw.head(10))
print(X_test_raw.head(10))
print('*' * 10)


                        Name Platform  Year_of_Release         Genre  \
0                 Wii Sports      Wii           2006.0        Sports   
1          Super Mario Bros.      NES           1985.0      Platform   
2             Mario Kart Wii      Wii           2008.0        Racing   
3          Wii Sports Resort      Wii           2009.0        Sports   
4   Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing   
5                     Tetris       GB           1989.0        Puzzle   
6      New Super Mario Bros.       DS           2006.0      Platform   
7                   Wii Play      Wii           2006.0          Misc   
8  New Super Mario Bros. Wii      Wii           2009.0      Platform   
9                  Duck Hunt      NES           1984.0       Shooter   

  Publisher  NA_Sales  EU_Sales  Other_Sales  Global_Sales  Critic_Score  \
0  Nintendo     41.36     28.96         8.45         82.53          76.0   
1  Nintendo     29.08      3.58         0.77         40

In [3]:
print("X_all")
print(X_all.head(10))

X_all
                        Name Platform  Year_of_Release         Genre  \
0                 Wii Sports      Wii           2006.0        Sports   
1          Super Mario Bros.      NES           1985.0      Platform   
2             Mario Kart Wii      Wii           2008.0        Racing   
3          Wii Sports Resort      Wii           2009.0        Sports   
4   Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing   
5                     Tetris       GB           1989.0        Puzzle   
6      New Super Mario Bros.       DS           2006.0      Platform   
7                   Wii Play      Wii           2006.0          Misc   
8  New Super Mario Bros. Wii      Wii           2009.0      Platform   
9                  Duck Hunt      NES           1984.0       Shooter   

  Publisher  NA_Sales  EU_Sales  Other_Sales  Global_Sales  Critic_Score  \
0  Nintendo     41.36     28.96         8.45         82.53          76.0   
1  Nintendo     29.08      3.58         0.77     

In [4]:
import re
import pandas as pd

EDITION_RE = re.compile(
    r"\b(remaster(ed)?|hd|definitive|ultimate|complete|collector'?s|"
    r"game of the year|goty|gold|deluxe|premium|special|limited|edition|"
    r"director'?s cut|anniversary|bundle|collection)\b",
    flags=re.IGNORECASE
)

ROMAN_RE = re.compile(r"\b(i{1,3}|iv|v|vi{0,3}|ix|x|xi|xii|xiii|xiv|xv)\b", flags=re.IGNORECASE)

def normalize_name(s: pd.Series) -> pd.Series:
    s = s.astype("string").fillna("__MISSING__").str.lower()
    # unify separators
    s = s.str.replace(r"[™®©]", "", regex=True)
    s = s.str.replace(r"[\(\)\[\]\{\}]", " ", regex=True)
    s = s.str.replace(r"[/:;,\.\!\?\|\\]", " ", regex=True)
    s = s.str.replace(r"[-_]+", " ", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s.replace("", "__MISSING__")

def split_base(s: pd.Series) -> pd.Series:
    # base before ':' or long dash patterns (common subtitle separators)
    s2 = s.str.replace(r"\s*:\s*", " : ", regex=True)
    # split on ":" or " - " (keep left)
    base = s2.str.split(r"\s:\s|\s-\s|\s—\s", n=1, expand=True)[0]
    base = base.str.strip()
    return base.replace("", "__MISSING__")

def franchise_key(s: pd.Series) -> pd.Series:
    s = s.copy()
    s = s.str.replace(EDITION_RE, " ", regex=True)
    s = s.str.replace(ROMAN_RE, " ", regex=True)
    s = s.str.replace(r"\b\d+\b", " ", regex=True)          # sequel numbers / years in title
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s.replace("", "__MISSING__")

# usage (на X_all)
if "Name" in X_all.columns:
    X_all["Name_norm"] = normalize_name(X_all["Name"])
    X_all["Name_base"] = split_base(X_all["Name_norm"])
    X_all["Franchise_key"] = franchise_key(X_all["Name_base"])


def add_name_flags(df):
    s = df["Name_norm"].astype("string")
    df["name_len"] = s.str.len().fillna(0).astype(int)
    df["name_words"] = s.str.split().str.len().fillna(0).astype(int)
    df["has_colon_or_dash"] = s.str.contains(r"\s:\s|\s-\s|\s—\s", regex=True).astype(int)
    df["has_digit"] = s.str.contains(r"\d").astype(int)
    df["has_roman"] = s.str.contains(ROMAN_RE, regex=True).astype(int)
    df["has_edition_word"] = s.str.contains(EDITION_RE, regex=True).astype(int)
    return df

if "Name_norm" in X_all.columns:
    X_all = add_name_flags(X_all)


/tmp/ipykernel_24147/2277753913.py:52: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_roman"] = s.str.contains(ROMAN_RE, regex=True).astype(int)
/tmp/ipykernel_24147/2277753913.py:53: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_edition_word"] = s.str.contains(EDITION_RE, regex=True).astype(int)


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

def add_oof_mean_count_features(
    X: pd.DataFrame,
    y,
    X_test: pd.DataFrame,
    keys,
    n_splits=5,
    seed=42,
    alpha=5.0,          # smoothing strength; 0 = без сглаживания
    fill_value="__MISSING__"
):
    """
    Добавляет OOF признаки:
      - <key>__jp_mean  : OOF mean (optionally smoothed)
      - <key>__jp_cnt   : OOF count
    и для test — статистики по full train.
    """
    if isinstance(y, (pd.Series, pd.DataFrame)):
        y = np.asarray(y).reshape(-1)
    else:
        y = np.asarray(y).reshape(-1)

    assert len(X) == len(y), "X and y must have same length"

    X = X.reset_index(drop=True).copy()
    X_test = X_test.reset_index(drop=True).copy()

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    global_mean = float(np.mean(y))

    def _compute_for_key(key_col: str):
        # гарантируем отсутствие NA в ключе
        X[key_col] = X[key_col].astype("string").fillna(fill_value)
        X_test[key_col] = X_test[key_col].astype("string").fillna(fill_value)

        oof_mean = np.full(len(X), global_mean, dtype=float)
        oof_cnt  = np.zeros(len(X), dtype=float)

        for tr_idx, va_idx in kf.split(X):
            tr_keys = X.loc[tr_idx, key_col].values

            stats = (
                pd.DataFrame({key_col: tr_keys, "y": y[tr_idx]})
                .groupby(key_col)["y"]
                .agg(["mean", "count"])
            )

            va_keys = X.loc[va_idx, key_col]
            m = va_keys.map(stats["mean"])
            c = va_keys.map(stats["count"])

            m = m.fillna(global_mean).astype(float).values
            c = c.fillna(0).astype(float).values

            if alpha and alpha > 0:
                # smoothed mean: (m*c + global*alpha)/(c+alpha)
                m = (m * c + global_mean * alpha) / (c + alpha)

            oof_mean[va_idx] = m
            oof_cnt[va_idx]  = c

        # full-train stats for test
        full_stats = (
            pd.DataFrame({key_col: X[key_col].values, "y": y})
            .groupby(key_col)["y"]
            .agg(["mean", "count"])
        )

        te_m = X_test[key_col].map(full_stats["mean"]).fillna(global_mean).astype(float).values
        te_c = X_test[key_col].map(full_stats["count"]).fillna(0).astype(float).values
        if alpha and alpha > 0:
            te_m = (te_m * te_c + global_mean * alpha) / (te_c + alpha)

        X[f"{key_col}__jp_mean"] = oof_mean
        X[f"{key_col}__jp_cnt"]  = oof_cnt
        X_test[f"{key_col}__jp_mean"] = te_m
        X_test[f"{key_col}__jp_cnt"]  = te_c

    for k in keys:
        if k in X.columns and k in X_test.columns:
            _compute_for_key(k)
        else:
            print(f"SKIP '{k}': not present in both X and X_test")

    return X, X_test


In [6]:
# PREPROCESS ON ALL DATA (train+test), THEN SPLIT BACK
# =========================

def make_ohe(min_freq: int):
    """Совместимость со sklearn: sparse_output (новый) vs sparse (старый)."""
    try:
        return OneHotEncoder(handle_unknown="ignore", min_frequency=min_freq, sparse_output=True)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", min_frequency=min_freq, sparse=True)

def preprocess_all_data(
    X_all_df: pd.DataFrame,
    text_cols,
    cat_cols,
    num_cols,
    min_freq: int = 50,
    tfidf_max_features: int = 80000,
):
    mats = []

    # 1) NUM
    if len(num_cols) > 0:
        num_imputer = SimpleImputer(strategy="median")
        num_scaler = StandardScaler(with_mean=False)
        num_data = num_scaler.fit_transform(num_imputer.fit_transform(X_all_df[num_cols]))
        mats.append(sparse.csr_matrix(num_data))
        print(f"NUM: {len(num_cols)} cols -> {num_data.shape}")

    # 2) CAT (ИСПРАВЛЕНО!)
    if len(cat_cols) > 0:
        cat_df = X_all_df[cat_cols].fillna("__MISSING__").astype(str)
        ohe = make_ohe(min_freq)
        cat_ohe = ohe.fit_transform(cat_df)
        mats.append(cat_ohe.tocsr())
        print(f"CAT: {len(cat_cols)} cols -> OHE shape {cat_ohe.shape}")

    # 3) TEXT (TF-IDF)
    def _flatten_1d(x):
        arr = np.asarray(x).ravel().astype(str)
        return np.where((arr == 'nan') | (arr == 'None') | (arr == '<NA>'), '', arr)

    for c in text_cols:
        # ИСПРАВЛЕНИЕ: fillna ПЕРЕД astype
        text_data = X_all_df[c].fillna("").astype(str).values
        text_data = _flatten_1d(text_data.reshape(-1, 1))
        
        tfidf = TfidfVectorizer(
            analyzer="char",
            ngram_range=(3, 5),
            min_df=2,
            max_features=tfidf_max_features,
        )
        tfidf_mat = tfidf.fit_transform(text_data)
        mats.append(tfidf_mat.tocsr())
        print(f"TEXT '{c}': TF-IDF shape {tfidf_mat.shape}, vocab={len(tfidf.vocabulary_)}")

    # stack
    X_proc = sparse.hstack(mats, format="csr")
    print(f"TOTAL features: {X_proc.shape[1]}")
    return X_proc

# Выбираем колонки
text_cols = [c for c in ["Name", "Name_root"] if c in X_all.columns]

cat_cols_all = X_all.select_dtypes(include=["object", "category", "bool", "string"]).columns.tolist()
cat_cols = [c for c in cat_cols_all if c not in set(text_cols)]

num_cols = [c for c in X_all.columns if c not in set(cat_cols) and c not in set(text_cols)]

print("text_cols:", text_cols)
print("cat_cols:", cat_cols)
print("num_cols:", num_cols)

X_all_proc = preprocess_all_data(
    X_all,
    text_cols=text_cols,
    cat_cols=cat_cols,
    num_cols=num_cols,
    min_freq=50,
    tfidf_max_features=80000,
)

n_train = len(X_train_raw)
X_train_proc = X_all_proc[:n_train]
X_test_proc  = X_all_proc[n_train:]

print("X_train_proc:", X_train_proc.shape, "X_test_proc:", X_test_proc.shape)


text_cols: ['Name']
cat_cols: ['Platform', 'Genre', 'Publisher', 'User_Score', 'Developer', 'Rating', 'Name_norm', 'Name_base', 'Franchise_key']
num_cols: ['Year_of_Release', 'NA_Sales', 'EU_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Count', 'name_len', 'name_words', 'has_colon_or_dash', 'has_digit', 'has_roman', 'has_edition_word']
NUM: 14 cols -> (21735, 14)
CAT: 9 cols -> OHE shape (21735, 200)
TEXT 'Name': TF-IDF shape (21735, 76512), vocab=76512
TOTAL features: 76726
X_train_proc: (16719, 76726) X_test_proc: (5016, 76726)


In [7]:
# =========================
# CV: RIDGE (on processed)
# =========================
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def cv_oof_ridge(X_proc, y_series: pd.Series, cv, alpha: float = 2.0):
    oof = np.zeros(len(y_series), dtype=float)
    fold_scores = []
    for fold, (tr_idx, va_idx) in enumerate(cv.split(X_proc, y_series), 1):
        m = Ridge(alpha=alpha)
        m.fit(X_proc[tr_idx], y_series.iloc[tr_idx])
        pred = np.clip(m.predict(X_proc[va_idx]), 0, None)
        oof[va_idx] = pred
        mae = float(mean_absolute_error(y_series.iloc[va_idx], pred))
        r = rmse(y_series.iloc[va_idx], pred)
        fold_scores.append((mae, r))
        print(f"[Ridge][fold {fold}] MAE={mae:.6f} RMSE={r:.6f}")
    maes = np.array([s[0] for s in fold_scores])
    rmses = np.array([s[1] for s in fold_scores])
    print(f"Ridge: MAE mean={maes.mean():.6f} std={maes.std():.6f} | RMSE mean={rmses.mean():.6f} std={rmses.std():.6f}")
    return oof, fold_scores

oof_ridge, ridge_scores = cv_oof_ridge(X_train_proc, y, cv, alpha=2.0)


[Ridge][fold 1] MAE=0.002759 RMSE=0.005082
[Ridge][fold 2] MAE=0.002634 RMSE=0.004881
[Ridge][fold 3] MAE=0.002831 RMSE=0.005253
[Ridge][fold 4] MAE=0.002787 RMSE=0.005156
[Ridge][fold 5] MAE=0.002700 RMSE=0.004971
Ridge: MAE mean=0.002742 std=0.000069 | RMSE mean=0.005069 std=0.000132


In [8]:
# =========================
# CV: CATBOOST (CPU only, without Name text cols)
# =========================
def make_catboost_frame(df: pd.DataFrame) -> pd.DataFrame:
    # CatBoost нормально принимает pandas.DataFrame. Для категорий лучше string + fillna.
    out = df.copy()
    for c in out.columns:
        if str(out[c].dtype) in ("object", "category", "string", "bool"):
            out[c] = out[c].astype("string").fillna("__MISSING__")
    return out

def cv_oof_catboost(X_df: pd.DataFrame, y_series: pd.Series, cv):
    # Удаляем текстовые Name фичи из CatBoost (их обрабатывает Ridge)
    drop_cols = [c for c in ["Name", "Name_root"] if c in X_df.columns]
    X_cb = X_df.drop(columns=drop_cols).reset_index(drop=True)
    X_cb = make_catboost_frame(X_cb)

    cat_cols = X_cb.select_dtypes(include=["object","category","string","bool"]).columns.tolist()
    cat_idx  = [X_cb.columns.get_loc(c) for c in cat_cols]

    oof = np.zeros(len(y_series), dtype=float)
    fold_scores = []
    best_iters = []

    for fold, (tr_idx, va_idx) in enumerate(cv.split(X_cb, y_series), 1):
        X_tr, X_va = X_cb.iloc[tr_idx], X_cb.iloc[va_idx]
        y_tr, y_va = y_series.iloc[tr_idx], y_series.iloc[va_idx]

        model = CatBoostRegressor(
            loss_function="MAE",
            eval_metric="MAE",
            iterations=6000,
            learning_rate=0.03,
            depth=8,
            l2_leaf_reg=6.0,
            random_seed=RANDOM_STATE,
            # CPU only (стабильнее, без требований к CUDA)
            task_type="CPU",
            # регуляризация
            subsample=0.8,
            rsm=0.8,
            bootstrap_type="Bernoulli",
            # контроль
            verbose=200,
        )

        model.fit(
            X_tr, y_tr,
            cat_features=cat_idx,
            eval_set=(X_va, y_va),
            use_best_model=True,
            early_stopping_rounds=500,
        )

        pred = np.clip(model.predict(X_va), 0, None)
        oof[va_idx] = pred

        mae = float(mean_absolute_error(y_va, pred))
        r = rmse(y_va, pred)
        fold_scores.append((mae, r))
        best_iters.append(int(model.get_best_iteration()))
        print(f"[CatBoost][fold {fold}] MAE={mae:.6f} RMSE={r:.6f} best_iter={best_iters[-1]}")

    maes = np.array([s[0] for s in fold_scores])
    rmses = np.array([s[1] for s in fold_scores])
    print(f"CatBoost: MAE mean={maes.mean():.6f} std={maes.std():.6f} | RMSE mean={rmses.mean():.6f} std={rmses.std():.6f}")

    return oof, fold_scores, best_iters

oof_cb, cb_scores, cb_best_iters = cv_oof_catboost(X_train_raw, y, cv)


0:	learn: 0.0755420	test: 0.0826102	best: 0.0826102 (0)	total: 58.7ms	remaining: 5m 52s
200:	learn: 0.0290304	test: 0.0323422	best: 0.0323422 (200)	total: 2.34s	remaining: 1m 7s
400:	learn: 0.0208751	test: 0.0262431	best: 0.0262431 (400)	total: 4.76s	remaining: 1m 6s
600:	learn: 0.0167465	test: 0.0236797	best: 0.0236797 (600)	total: 7.03s	remaining: 1m 3s
800:	learn: 0.0137924	test: 0.0223172	best: 0.0223172 (800)	total: 11.8s	remaining: 1m 16s
1000:	learn: 0.0121560	test: 0.0217573	best: 0.0217573 (1000)	total: 17.8s	remaining: 1m 28s
1200:	learn: 0.0109160	test: 0.0213974	best: 0.0213974 (1200)	total: 22.9s	remaining: 1m 31s
1400:	learn: 0.0099390	test: 0.0211208	best: 0.0211208 (1400)	total: 27.4s	remaining: 1m 29s
1600:	learn: 0.0091809	test: 0.0209107	best: 0.0209107 (1600)	total: 30.3s	remaining: 1m 23s
1800:	learn: 0.0085450	test: 0.0207766	best: 0.0207695 (1748)	total: 33s	remaining: 1m 17s
2000:	learn: 0.0079716	test: 0.0206086	best: 0.0206085 (1998)	total: 35.6s	remaining: 1m

In [9]:
# =========================
# ENSEMBLE WEIGHT (grid search on OOF)
# =========================
weights = np.linspace(0, 1, 201)  # w = доля CatBoost
best = {"w": None, "mae": np.inf, "rmse": np.inf}

for w in weights:
    ens = w * oof_cb + (1 - w) * oof_ridge
    mae = float(mean_absolute_error(y, ens))
    r = rmse(y, ens)
    if mae < best["mae"]:
        best = {"w": float(w), "mae": mae, "rmse": r}

print("Best ensemble:", best)


Best ensemble: {'w': 0.0, 'mae': 0.0027422913654672246, 'rmse': 0.005070518376312814}


In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

def _prep_lookup_keys(df: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame(index=df.index)

    if "Name_norm" in df.columns:
        out["Name_norm"] = df["Name_norm"].astype("string").fillna("__MISSING__")
    else:
        s = df["Name"].astype("string").fillna("")
        s = s.str.lower()
        s = s.str.replace(r"[\u2122\u00ae]", "", regex=True)     # ™ ®
        s = s.str.replace(r"[^0-9a-z]+", " ", regex=True)        # латиница+цифры
        s = s.str.replace(r"\s+", " ", regex=True).str.strip()
        out["Name_norm"] = s.replace("", "__MISSING__")

    out["Platform"] = df["Platform"].astype("string").fillna("__MISSING__")

    year = pd.to_numeric(df["Year_of_Release"], errors="coerce").fillna(-1).astype(int)
    out["Year_of_Release"] = year

    return out


def build_lookup(train_df: pd.DataFrame, y: pd.Series, key_cols, min_cnt: int = 1) -> pd.Series:
    """
    Возвращает pd.Series с MultiIndex по key_cols и значениями median(y).
    min_cnt позволяет отфильтровать редкие ключи (полезно на fallback уровнях).
    """
    keys = _prep_lookup_keys(train_df)[key_cols]
    tmp = keys.copy()
    tmp["y"] = y.values

    g = tmp.groupby(key_cols)["y"].agg(["median", "count"])
    g = g[g["count"] >= min_cnt]
    return g["median"]

def apply_lookup(lookup: pd.Series, df: pd.DataFrame, key_cols):
    keys = _prep_lookup_keys(df)[key_cols]
    mi = pd.MultiIndex.from_frame(keys)
    pred = lookup.reindex(mi).to_numpy()
    found = ~pd.isna(pred)
    return pred, found

def oof_lookup_pred(df: pd.DataFrame, y: pd.Series, cv, key_cols, min_cnt: int = 1):
    """
    OOF-lookup: на каждом фолде строим lookup только по train-fold,
    применяем к val-fold.
    """
    oof = np.full(len(df), np.nan, dtype=float)

    for fold, (tr_idx, va_idx) in enumerate(cv.split(df), 1):
        lookup = build_lookup(df.iloc[tr_idx], y.iloc[tr_idx], key_cols, min_cnt=min_cnt)
        pred_va, found_va = apply_lookup(lookup, df.iloc[va_idx], key_cols)
        oof[va_idx] = pred_va  # NaN там, где не найдено

        cov = float(np.mean(found_va))
        # MAE только по найденным
        if np.any(found_va):
            mae = mean_absolute_error(y.iloc[va_idx].values[found_va], pred_va[found_va])
        else:
            mae = np.nan
        print(f"[OOF-LOOKUP][fold {fold}] coverage={cov:.2%}, MAE(found-only)={mae}")

    return oof


In [ ]:
# =========================
# FINAL FIT + PREDICT TEST
# =========================
# Ridge final on all processed train
ridge_final = Ridge(alpha=2.0)
ridge_final.fit(X_train_proc, y)
pred_ridge = np.clip(ridge_final.predict(X_test_proc), 0, None)

# CatBoost final: fit with holdout for early stopping
drop_cols = [c for c in ["Name", "Name_root"] if c in X_train_raw.columns]
X_cb_full = X_train_raw.drop(columns=drop_cols).reset_index(drop=True)
X_cb_test = X_test_raw.drop(columns=drop_cols).reset_index(drop=True)
# FIX
for df in (X_cb_full, X_cb_test):
    if "Global_Sales" in df.columns:
        df.drop(columns=["Global_Sales"], inplace=True)

X_cb_full = make_catboost_frame(X_cb_full)
X_cb_test = make_catboost_frame(X_cb_test)

cat_cols = X_cb_full.select_dtypes(include=["object","category","string","bool"]).columns.tolist()

for c in cat_cols:
    X_cb_full[c] = X_cb_full[c].astype("string").fillna("__MISSING__")
    X_cb_test[c] = X_cb_test[c].astype("string").fillna("__MISSING__")

# 3) после приведения считаем индексы
cat_idx = [X_cb_full.columns.get_loc(c) for c in cat_cols]

cat_cols = X_cb_full.select_dtypes(include=["object","category","string","bool"]).columns.tolist()
cat_idx  = [X_cb_full.columns.get_loc(c) for c in cat_cols]

X_tr, X_va, y_tr, y_va = train_test_split(
    X_cb_full, y, test_size=0.15, random_state=RANDOM_STATE
)



cb_final = CatBoostRegressor(
    loss_function="MAE",
    eval_metric="MAE",
    iterations=20000,

    # best_params (Optuna best)
    bootstrap_type="Bernoulli",
    grow_policy="Depthwise",
    learning_rate=0.005336574757662616,
    depth=11,
    l2_leaf_reg=47.22898537933602,
    rsm=0.9069506230060724,
    min_data_in_leaf=183,
    random_strength=10.226847977218757,
    one_hot_max_size=26,
    max_ctr_complexity=7,
    border_count=177,
    leaf_estimation_method="Exact",
    leaf_estimation_iterations=7,
    subsample=0.687791476319972,
    model_size_reg=0.93981939840869,
    feature_border_type="GreedyLogSum",
    score_function="L2",

    # fixed / infra
    random_seed=RANDOM_STATE,
    task_type="CPU",
    verbose=200,
)


cb_final.fit(
    X_tr, y_tr,
    cat_features=cat_idx,
    eval_set=(X_va, y_va),
    use_best_model=True,
    early_stopping_rounds=500,
)

pred_cb = np.clip(cb_final.predict(X_cb_test), 0, None)

w = best["w"] if best["w"] is not None else 0.5
pred_ens = np.clip(w * pred_cb + (1 - w) * pred_ridge, 0, None)

print(f"Pred stats: ridge_mean={pred_ridge.mean():.4f}, cb_mean={pred_cb.mean():.4f}, ens_mean={pred_ens.mean():.4f}, w={w}")


# =========================
# LOOKUP (train -> test)
# =========================
# Уровень 1: строгий ключ (Name_norm + Platform + Year)
key1 = ["Name_norm", "Platform", "Year_of_Release"]
lk1 = build_lookup(X_train_raw, y, key1, min_cnt=1)
lk1_pred, lk1_found = apply_lookup(lk1, X_test_raw, key1)

pred_final = pred_ens.copy()
pred_final[lk1_found] = lk1_pred[lk1_found]

print(f"Lookup L1 hit-rate: {lk1_found.mean():.2%} ({lk1_found.sum()}/{len(lk1_found)})")

# (опционально) Уровень 2: fallback без года (иногда даёт сильный буст, но аккуратно)
# Если включаете — лучше поставить min_cnt>=2, чтобы уменьшить шум.
key2 = ["Name_norm", "Platform"]
lk2 = build_lookup(X_train_raw, y, key2, min_cnt=2)
lk2_pred, lk2_found = apply_lookup(lk2, X_test_raw, key2)

# применяем fallback ТОЛЬКО там, где L1 не нашёлся
need2 = (~lk1_found) & (lk2_found)
pred_final[need2] = lk2_pred[need2]

print(f"Lookup L2 extra hit-rate: {need2.mean():.2%} (+{need2.sum()} rows)")

pred_final = np.clip(pred_final, 0, None)

print(f"Final stats: ens_mean={pred_ens.mean():.4f}, final_mean={pred_final.mean():.4f}")

# Оценка качества на тренировочных данных
y_pred_train = cb_final.predict(X_tr)
print('Mean Absolute Error = ', mean_absolute_error(y_tr, y_pred_train))
print('Mean Absolute Percentage Error = ', mean_absolute_percentage_error(y_tr, y_pred_train))


0:	learn: 0.0763669	test: 0.0840482	best: 0.0840482 (0)	total: 20.9ms	remaining: 10m 26s
200:	learn: 0.0622395	test: 0.0695123	best: 0.0695123 (200)	total: 2.78s	remaining: 6m 52s
400:	learn: 0.0579286	test: 0.0647283	best: 0.0647283 (400)	total: 5.54s	remaining: 6m 48s
600:	learn: 0.0560376	test: 0.0623129	best: 0.0623129 (600)	total: 8.29s	remaining: 6m 45s
800:	learn: 0.0546627	test: 0.0606705	best: 0.0606705 (800)	total: 11.8s	remaining: 7m 8s
1000:	learn: 0.0536187	test: 0.0595491	best: 0.0595491 (1000)	total: 16.2s	remaining: 7m 48s
1200:	learn: 0.0527798	test: 0.0588002	best: 0.0588002 (1200)	total: 20.1s	remaining: 8m 2s
1400:	learn: 0.0521236	test: 0.0581229	best: 0.0581229 (1400)	total: 22.9s	remaining: 7m 46s
1600:	learn: 0.0515760	test: 0.0576463	best: 0.0576463 (1600)	total: 26s	remaining: 7m 40s
1800:	learn: 0.0510697	test: 0.0571808	best: 0.0571808 (1800)	total: 29.1s	remaining: 7m 35s
2000:	learn: 0.0504660	test: 0.0566412	best: 0.0566409 (1999)	total: 32.1s	remaining: 

KeyboardInterrupt: 

In [ ]:
# =========================
# SUBMISSION
# =========================
sub = pd.DataFrame({"Id": test_ids, "JP_Sales": pred_ens})
sub["JP_Sales"] = sub["JP_Sales"].round(2)
sub.to_csv("sub_external.csv", index=False)
sub.head(10)


,Id,JP_Sales
0,1,0.07
1,2,0.00
2,3,0.00
3,4,0.00
4,5,0.00
5,6,0.01
6,7,0.01
7,8,0.12
8,9,3.37
9,10,0.00
